In [2]:
from collections import Counter, defaultdict
from datetime import date, timedelta
import pandas as pd

In [3]:
start_date = 20150830 #Sun #actual start_date = 20150902
end_date = 20160109 #Sat #actual end_date = 20151231

In [4]:
#make a list of all Sundays within analysis time period
sundays = []
s = date(int(str(start_date)[:4]),int(str(start_date)[4:6]),int(str(start_date)[6:8]))
s_end = date(int(str(end_date)[:4]),int(str(end_date)[4:6]),int(str(end_date)[6:8]))
while s <= s_end:        
    sundays.append(s.strftime("%Y%m%d"))
    s += timedelta(days = 7)
    
print "Number of days: {}".format(len(sundays))
sundays

Number of days: 19


['20150830',
 '20150906',
 '20150913',
 '20150920',
 '20150927',
 '20151004',
 '20151011',
 '20151018',
 '20151025',
 '20151101',
 '20151108',
 '20151115',
 '20151122',
 '20151129',
 '20151206',
 '20151213',
 '20151220',
 '20151227',
 '20160103']

In [22]:
#SID|first tp date|first tp type|weekly touchpoint count|
pre_holiday_purchase = defaultdict(set)
holiday_multi_purc = defaultdict(set)
past_sid = None
first_tp = {}
weekly_tp = Counter()
with open("holiday_purchasers_20150902_20151231.txt", 'r') as fh:
    for line in fh:
        v = line.strip().split("|")
        current_sid = v[0]
        timestamp = v[1]
        lines = [2]
        event_type = v[3]
        
        #first touchpoint date and type
        if current_sid <> past_sid:
            first_tp[current_sid] = timestamp[:8] + "|" + event_type
        past_sid = current_sid
        
        if event_type in ["E", "P"]: 
            #purchase prior to 20151127
            if int(timestamp[:8]) < 20151127:
                    pre_holiday_purchase[current_sid].add(timestamp)
            #purchase from 20151127 - 20151231
            else:
                holiday_multi_purc[current_sid].add(timestamp)
        #weekly touchpoint count
        elif event_type not in ["E", "P"] :
            for i in range(len(sundays)-1):
                if int(sundays[i]) <= int(timestamp[:8]) < int(sundays[i+1]):
                    weekly_tp[current_sid + "|" + sundays[i]] += 1
        else:
            print line.strip()

In [14]:
# output weekly touchpoint count in long form
fh_out = open("tmp_out.txt", 'w')
for i in weekly_tp:
    print >> fh_out, "{}|{}".format(i,weekly_tp[i])
fh_out.close()    

In [15]:
# reshape weekly touchpoint count - long to wide
df = pd.read_csv("tmp_out.txt", delimiter = "|", header = None)
df.head()

,0,1,2
0,1030335361,20151122,37
1,372274566,20151122,4
2,1007287625,20151108,2
3,1030335361,20151129,21
4,1875504932,20151129,1


In [16]:
# weekly touchpoint count in wide form
df_wide = df.pivot(index = 0, columns = 1, values = 2)
df_wide.to_csv("wide_tmp_out.txt",delimiter = '|,',index = True)

In [23]:
fh_out = open("tmp_out_2.txt",'w')
for i in first_tp:
    if i in pre_holiday_purchase:
        pre_holiday_purchase_cnt = len(pre_holiday_purchase[i])
    else:
        pre_holiday_purchase_cnt = 0
    if i in holiday_multi_purc:
        holiday_multi_purc_cnt = len(holiday_multi_purc[i])
    else:
        holiday_multi_purc_cnt = 0
    print >> fh_out, "{}|{}|{}|{}".format(i, first_tp[i],pre_holiday_purchase_cnt,holiday_multi_purc_cnt)
fh_out.close()    